In [2]:
'''
https://zenodo.org/records/6382090
import zipfile
import os

# Descomprimir el dataset
with zipfile.ZipFile('wm_barriers_data.zip', 'r') as zip_ref:
    zip_ref.extractall('wm_barriers_data')
'''

In [ ]:
'''
import xmltodict
import json
import os

def convert_cvat_xml_to_json(xml_file):
    with open(xml_file) as f:
        xml_content = f.read()
    
    annotations = xmltodict.parse(xml_content)

    data = []

    # Extraer imágenes y sus anotaciones
    try:
        images = annotations['annotations']['image']
        if isinstance(images, list):
            for image in images:
                image_name = image['@name']
                image_annotations = []

                # Asegúrate de que 'box' sea una lista
                boxes = image.get('box', [])
                if isinstance(boxes, list):
                    for box in boxes:
                        label = box.get('@label', None)

                        # Extraer atributos
                        attributes = {}
                        attrs = box.get('attribute', [])
                        if isinstance(attrs, list):
                            for attr in attrs:
                                attr_name = attr['@name'] if '@name' in attr else None
                                attr_value = attr.text if hasattr(attr, 'text') and attr.text else None
                                if attr_name:
                                    attributes[attr_name] = attr_value
                        else:
                            # Si solo hay un atributo, maneja eso
                            attr_name = attrs['@name'] if '@name' in attrs else None
                            attr_value = attrs.text if hasattr(attrs, 'text') and attrs.text else None
                            if attr_name:
                                attributes[attr_name] = attr_value

                        if label is not None:
                            image_annotations.append({
                                'label': label,
                                'attributes': attributes
                            })

                data.append({
                    'image': image_name,
                    'annotations': image_annotations
                })
        else:
            print("No images found or images is not a list.")

    except Exception as e:
        print(f'Error processing the XML: {e}')

    # Ordenar data por el nombre de la imagen (asegúrate de que los nombres sean compatibles)
    data.sort(key=lambda x: x['image'])

    return data

# Convertir y guardar como JSON
json_data = convert_cvat_xml_to_json('wm_barriers_data/wm_annotations.xml')

# Guardar en un archivo JSON
with open('annotations.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# Comprobar el contenido del JSON
if not json_data:
    print("No annotations found. JSON data is empty.")
else:
    print(f"Converted {len(json_data)} images with annotations.")

'''


In [ ]:

'''
import json
import cv2
import numpy as np
import os

# Cargar datos desde el archivo JSON
with open('annotations.json', 'r') as json_file:
    annotations = json.load(json_file)

# Inicializar listas para las imágenes y etiquetas
images = []
numeric_labels = []

# Mapeo de etiquetas a valores numéricos
label_mapping = {
    'step': 0,         # 0 - no accesible
    'stair': 0,        # 0 - no accesible
    'ramp': 1,         # 1 - accesible
    'grab_bar': 0      # 0 - no accesible
}

IMG_HEIGHT, IMG_WIDTH = 224, 224  # Cambia según tu modelo

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: No se pudo cargar la imagen en {image_path}")
        return None
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img / 255.0  # Normalizar a [0,1]
    return img

# Cargar imágenes y etiquetas
for annotation in annotations:
    image_path = os.path.join("wm_barriers_data", annotation['image'])
    img = preprocess_image(image_path)
    
    if img is not None:
        # Si no hay anotaciones, asignar 1
        if not annotation['annotations']:
            images.append(img)
            numeric_labels.append(1)  # Etiqueta 1 para accesible (sin anotaciones)
        else:
            # Si hay anotaciones, revisar si hay una anotación 'ramp'
            is_ramp = any(ann['label'] == 'ramp' for ann in annotation['annotations'])
            if is_ramp:
                images.append(img)
                numeric_labels.append(1)  # Etiqueta 1 para accesible (ramp)
            else:
                images.append(img)
                numeric_labels.append(0)  # Etiqueta 0 para no accesible (cualquier otra cosa)

# Convertir listas a arrays numpy
images = np.array(images)
numeric_labels = np.array(numeric_labels)

# Comprobar longitudes de los arrays
print(f"Total de imágenes procesadas: {len(images)}")
print(f"Total de etiquetas procesadas: {len(numeric_labels)}")

# Guardar los datos procesados
if len(images) > 0 and len(numeric_labels) > 0:
    np.savez_compressed('processed_data_full.npz', 
                        images=images, 
                        labels=numeric_labels)
    print("Datos guardados en 'processed_data_full.npz'")
else:
    print("No hay datos suficientes para guardar.")
'''

In [ ]:
import numpy as np

# Cargar el archivo .npz desde Google Drive
data = np.load('processed_data_full.npz')

# Acceder a las matrices guardadas en el archivo
images = data['images']
numeric_labels = data['labels']

X = images  
y = numeric_labels


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(images, numeric_labels, test_size=0.3, random_state=42)
print(f"Datos divididos: {len(X_train)} entrenamientos y {len(X_val)} validaciones.")

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Definir el tamaño de las imágenes
IMG_HEIGHT = 224  
IMG_WIDTH = 224   


# Definir el modelo
def create_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    
    # Primer bloque de convolución
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Segundo bloque de convolución
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Tercer bloque de convolución
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Cuarto bloque de convolución
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    
    # Capas completamente conectadas
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid')) 
    
    return model


model = create_model()

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',  
              metrics=['accuracy'])

# Imprimir un resumen del modelo
model.summary()


In [ ]:
# Número total de ejemplos
total = len(numeric_labels)

# Número de ejemplos por clase
no_accesible = np.sum(numeric_labels == 0)
accesible = np.sum(numeric_labels == 1)

# Inverso de la frecuencia para penalizar más la clase minoritaria
weight_for_0 = (1 / no_accesible) * (total / 2.0)
weight_for_1 = (1 / accesible) * (total / 2.0)

# Mostrar los pesos calculados
print(f"Peso para la clase 'No Accesible' (Clase 0): {weight_for_0}")
print(f"Peso para la clase 'Accesible' (Clase 1): {weight_for_1}")

# Asignar los pesos en la forma de un diccionario
class_weights = {0: weight_for_0, 1: weight_for_1}

# Entrenar el modelo
model.fit(X_train, y_train, class_weight=class_weights, batch_size=128, epochs=5, validation_data=(X_val, y_val))
model.save('accessibility_model.keras')

In [ ]:
model.save('accessibility_model.keras')


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Realizar predicciones en el conjunto de validación (X_val)
y_pred = model.predict(X_val)

# Las predicciones serán probabilidades, así que redondeamos para obtener 0 o 1
y_pred_classes = np.round(y_pred).astype(int)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_val, y_pred_classes)

# Mostrar la matriz de confusión en consola
print("Matriz de Confusión:")
print(conf_matrix)

# Visualizar la matriz de confusión con seaborn
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['No Accesible', 'Accesible'], yticklabels=['No Accesible', 'Accesible'])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión')
plt.show()